In [1]:
import cv2
import numpy as np
import os
import mediapipe as mp
import pandas as pd
from fastdtw import fastdtw

# read train, test data

In [2]:
train = pd.read_csv("./dtw_sample_image/train_skeleton.csv", header=[0,1])

In [3]:
test = pd.read_csv("./dtw_sample_image/test_skeleton.csv", header=[0,1])

In [4]:
train.head()

playtime      NOSE                               LEFT_EYE_INNER            \
       sec         x         y         z         v              x         y   
0      1.0  0.530722  0.288515 -0.418072  0.999998       0.536021  0.274355   
1      2.0  0.531541  0.288517 -0.418771  0.999998       0.536534  0.274353   
2      3.0  0.531779  0.286608 -0.394940  0.999995       0.536731  0.272949   
3      4.0  0.532713  0.285407 -0.400416  0.999992       0.537325  0.271886   
4      5.0  0.533324  0.283762 -0.397750  0.999990       0.537871  0.270842   

                       LEFT_EYE  ... RIGHT_HEEL           LEFT_FOOT_INDEX  \
          z         v         x  ...          z         v               x   
0 -0.394486  0.999993  0.540506  ...   0.313760  0.885387        0.563856   
1 -0.394490  0.999992  0.540921  ...   0.398194  0.887266        0.564073   
2 -0.366800  0.999977  0.541114  ...   0.394549  0.888964        0.564106   
3 -0.371925  0.999959  0.541724  ...   0.379305  0.889909        0.564040   
4 -0.367384  0.999936  0.542315  ...   0.369387  0.891310        0.563863   

                                RIGHT_FOOT_INDEX                                
          y         z         v                x         y         z         v  
0  0.829309  0.148345  0.990084         0.507596  0.830114  0.134731  0.988936  
1  0.828517  0.252427  0.990203         0.507779  0.830136  0.222445  0.989117  
2  0.827820  0.262247  0.990189         0.507773  0.830136  0.215165  0.989273  
3  0.827544  0.244534  0.989954         0.507768  0.830144  0.196385  0.989313  
4  0.827439  0.222800  0.989689         0.507735  0.830162  0.185588  0.989340  

[5 rows x 133 columns]

In [5]:
test.head()

playtime      NOSE                               LEFT_EYE_INNER            \
       sec         x         y         z         v              x         y   
0      1.0  0.517440  0.191790 -0.356848  0.999980       0.525520  0.180241   
1      2.0  0.515312  0.191814 -0.262179  0.999981       0.524741  0.180255   
2      3.0  0.511461  0.192231 -0.286979  0.999977       0.522251  0.180483   
3      4.0  0.509108  0.193643 -0.367301  0.999971       0.518850  0.181364   
4      5.0  0.508706  0.197692 -0.395361  0.999965       0.517989  0.185085   

                       LEFT_EYE  ... RIGHT_HEEL           LEFT_FOOT_INDEX  \
          z         v         x  ...          z         v               x   
0 -0.308863  0.999951  0.531481  ...   0.344362  0.927483        0.588020   
1 -0.212149  0.999953  0.530590  ...   0.303045  0.927459        0.587197   
2 -0.239952  0.999946  0.528537  ...   0.297698  0.927382        0.587022   
3 -0.323281  0.999933  0.525608  ...   0.299199  0.925959        0.585917   
4 -0.350740  0.999922  0.524685  ...   0.288224  0.925201        0.585518   

                                RIGHT_FOOT_INDEX                                
          y         z         v                x         y         z         v  
0  0.871516  0.022288  0.973722         0.435271  0.850148  0.137319  0.976482  
1  0.869676  0.032775  0.974414         0.434729  0.851558  0.092860  0.977257  
2  0.865889  0.038175  0.974965         0.432286  0.852560  0.086364  0.977879  
3  0.860315  0.043411  0.975676         0.433114  0.851815  0.085901  0.978507  
4  0.854297  0.046811  0.976267         0.433030  0.851337  0.069414  0.979128  

[5 rows x 133 columns]

# extract x, y in raw data

In [6]:
train_xy = train[[(a,b) for a, b in train.columns if b in ['x','y']]]

In [7]:
test_xy = test[[(a,b )for a, b in test.columns if b in ['x', 'y']]]

In [8]:
train_xy.head()

NOSE           LEFT_EYE_INNER            LEFT_EYE            \
          x         y              x         y         x         y   
0  0.530722  0.288515       0.536021  0.274355  0.540506  0.274250   
1  0.531541  0.288517       0.536534  0.274353  0.540921  0.274250   
2  0.531779  0.286608       0.536731  0.272949  0.541114  0.272977   
3  0.532713  0.285407       0.537325  0.271886  0.541724  0.271931   
4  0.533324  0.283762       0.537871  0.270842  0.542315  0.270960   

  LEFT_EYE_OUTER           RIGHT_EYE_INNER            ... RIGHT_ANKLE  \
               x         y               x         y  ...           x   
0       0.545191  0.274156        0.518991  0.275286  ...    0.511483   
1       0.545709  0.274160        0.519665  0.275248  ...    0.511481   
2       0.545953  0.273080        0.519876  0.273632  ...    0.511337   
3       0.546528  0.272094        0.520972  0.272395  ...    0.511129   
4       0.547047  0.271220        0.521723  0.271190  ...    0.510768   

            LEFT_HEEL           RIGHT_HEEL           LEFT_FOOT_INDEX  \
          y         x         y          x         y               x   
0  0.784886  0.562543  0.793876   0.519781  0.798034        0.563856   
1  0.782006  0.562445  0.789262   0.519233  0.790564        0.564073   
2  0.781147  0.562497  0.787841   0.518558  0.789669        0.564106   
3  0.780958  0.562530  0.787342   0.517924  0.789782        0.564040   
4  0.780843  0.562514  0.787462   0.517047  0.789852        0.563863   

            RIGHT_FOOT_INDEX            
          y                x         y  
0  0.829309         0.507596  0.830114  
1  0.828517         0.507779  0.830136  
2  0.827820         0.507773  0.830136  
3  0.827544         0.507768  0.830144  
4  0.827439         0.507735  0.830162  

[5 rows x 66 columns]

In [9]:
test_xy.head()

NOSE           LEFT_EYE_INNER            LEFT_EYE            \
          x         y              x         y         x         y   
0  0.517440  0.191790       0.525520  0.180241  0.531481  0.180967   
1  0.515312  0.191814       0.524741  0.180255  0.530590  0.180986   
2  0.511461  0.192231       0.522251  0.180483  0.528537  0.181172   
3  0.509108  0.193643       0.518850  0.181364  0.525608  0.181895   
4  0.508706  0.197692       0.517989  0.185085  0.524685  0.185464   

  LEFT_EYE_OUTER           RIGHT_EYE_INNER            ... RIGHT_ANKLE  \
               x         y               x         y  ...           x   
0       0.538088  0.182000        0.506933  0.179594  ...    0.464793   
1       0.536555  0.182032        0.505676  0.179655  ...    0.465125   
2       0.534004  0.182185        0.502326  0.180036  ...    0.465270   
3       0.530920  0.182657        0.498691  0.181288  ...    0.465648   
4       0.529591  0.185836        0.497849  0.185010  ...    0.465786   

            LEFT_HEEL           RIGHT_HEEL           LEFT_FOOT_INDEX  \
          y         x         y          x         y               x   
0  0.796986  0.555255  0.839984   0.470715  0.820219        0.588020   
1  0.798281  0.555530  0.837718   0.471077  0.821383        0.587197   
2  0.798566  0.555894  0.834878   0.471470  0.822034        0.587022   
3  0.798527  0.556102  0.830296   0.472023  0.821977        0.585917   
4  0.799773  0.556113  0.827486   0.472339  0.824214        0.585518   

            RIGHT_FOOT_INDEX            
          y                x         y  
0  0.871516         0.435271  0.850148  
1  0.869676         0.434729  0.851558  
2  0.865889         0.432286  0.852560  
3  0.860315         0.433114  0.851815  
4  0.854297         0.433030  0.851337  

[5 rows x 66 columns]

# flatten data by frame

In [10]:
train_list = train_xy.values

In [11]:
test_list = test_xy.values

In [12]:
train_list

array([[0.53072172, 0.28851521, 0.53602087, ..., 0.82930911, 0.50759625,
        0.83011401],
       [0.53154135, 0.28851709, 0.53653449, ..., 0.82851696, 0.50777948,
        0.83013618],
       [0.53177887, 0.28660843, 0.53673083, ..., 0.82781965, 0.5077728 ,
        0.83013576],
       ...,
       [0.52776146, 0.28853187, 0.53469318, ..., 0.82671815, 0.507568  ,
        0.8295188 ],
       [0.52807343, 0.28867352, 0.53479505, ..., 0.82660395, 0.50701976,
        0.82944769],
       [0.52738917, 0.28881958, 0.53424871, ..., 0.82603091, 0.50701463,
        0.82916361]])

In [13]:
test_list

array([[0.51744032, 0.19179034, 0.52552009, ..., 0.87151581, 0.43527079,
        0.8501476 ],
       [0.51531196, 0.19181417, 0.52474076, ..., 0.86967587, 0.43472892,
        0.85155767],
       [0.5114609 , 0.19223098, 0.52225101, ..., 0.86588943, 0.43228585,
        0.85255998],
       ...,
       [0.49047166, 0.20908014, 0.50032771, ..., 0.84921831, 0.43841267,
        0.84010637],
       [0.49122792, 0.20719719, 0.50102711, ..., 0.85077542, 0.43907666,
        0.84147143],
       [0.49136463, 0.20585416, 0.50134426, ..., 0.85116839, 0.43951377,
        0.84170628]])

# fastdtw

In [14]:
dtw_results = [fastdtw(a,b)[0] for a, b in zip(train_list, test_list)]

In [15]:
dtw_results 

[8.296622402966022,
 8.036836087703705,
 6.5674562603235245,
 6.460594303905964,
 6.238546900451183,
 6.081121563911438,
 4.508420780301094,
 4.049132332205772,
 3.489009104669094,
 3.082674011588097,
 3.1557435244321823,
 3.259214609861374,
 3.334952987730503,
 3.480571001768112,
 3.471517615020275,
 3.2616143226623535,
 2.7994705587625504,
 2.4400724321603775,
 2.6938976496458054,
 2.6797751188278203,
 2.727222427725792,
 2.5934664011001587,
 2.453677073121071,
 2.306506499648094,
 2.2239078134298325,
 2.2291150838136673,
 2.412920966744423,
 2.5311928540468216,
 2.6117063164711,
 2.726651892066002]

In [16]:
np.array(dtw_results).mean()  # 동일한 동작을 했을 때 3.74 -> 높은 정확도일 때 3 ~ 4 정도의 dtw 값이 나옴

3.740120363235474

In [17]:
dtw_same = [fastdtw(a,b)[0] for a, b in zip(train_list, train_list)]

In [18]:
np.array(dtw_same).mean()  # 같은 영상 (완벽도가 100%일때) 0이 나옴

0.0

## fastdtw using error data

In [19]:
clap_clap = pd.read_csv("./error_sample_image/clap_clap_skeleton.csv", header=[0,1])

In [20]:
clap_clap_xy = clap_clap[[(a,b) for a, b in clap_clap.columns if b in ['x','y']]]

In [21]:
clap_clap_list = clap_clap_xy.values

In [22]:
foot_stamp_clap = pd.read_csv("./error_sample_image/foot_stamp_clap_skeleton.csv", header=[0,1])

In [23]:
foot_stamp_clap_xy = foot_stamp_clap[[(a,b) for a, b in foot_stamp_clap.columns if b in ['x','y']]]

In [24]:
foot_stamp_clap_list = foot_stamp_clap_xy.values

In [25]:
dtw_clap_clap = [fastdtw(a,b)[0] for a, b in zip(train_list, clap_clap_list)]

In [27]:
np.array(dtw_clap_clap).mean()

6.70230442037185

In [28]:
dtw_foot_stamp_clap = [fastdtw(a,b)[0] for a, b in zip(train_list, foot_stamp_clap_list)]

In [29]:
np.array(dtw_foot_stamp_clap).mean()

7.056086980799834

- 실제로 다른 동작의 데이터를 넣어봤을 때 6~7의 높은 dtw가 나오는 것을 보았을때 성능이 좋다고 판단할 수 있음

## fastdtw between error data
- clap_clap 과 foot_stamp_clap 은 모두 마지막에 박수치는 동작이 들어가기 때문에 dtw가 유사하다고 결론 내릴 수 있는지 확인 해보기

In [30]:
dtw_btw_errors = [fastdtw(a,b)[0] for a, b in zip(clap_clap_list, foot_stamp_clap_list)]

In [31]:
np.array(dtw_btw_errors).mean()

2.7357847720384596

- 완전히 같은 동작이 아니더라도 비슷한 동작을 했을때 또한 높은 유사도를 보여줌  
- 그러나 오히려 같은 동작을 했을 때 보다 높은 유사도를 보이기 때문에 이 부분은 해결이 필요함